In [1]:
import cv2
import numpy as np
import json
import socket
import threading
import time  # Import time module for tracking elapsed time


In [2]:
# Server details
SERVER_ADDRESS = 'DESKTOP-28SQ46K'  # Replace with the actual server address
SERVER_PORT = 8001
last_sent_message = ""

In [3]:
# Function to handle receiving messages from the server
def receive_messages(sock):
    while True:
        try:
            message = sock.recv(1024).decode('utf-8')
            if message:
                print(f"Received from server: {message}")
            else:
                break
        except Exception as e:
            print(f"Error receiving message: {e}")
        breaks

In [4]:
# Function to start the client and connect to the server
def start_client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        client_socket.connect((SERVER_ADDRESS, SERVER_PORT))
        print("Connected to the server.")

        # Start a thread to listen for messages from the server
        threading.Thread(target=receive_messages, args=(client_socket,), daemon=True).start()

        return client_socket

    except Exception as e:
        print(f"Failed to connect to the server: {e}")
        return None

In [5]:
# Function to send recognized name to the server
def send_recognized_name(name, client_socket):
    message = ""
    if client_socket:
        try:
            message =f"SENDTO:1|RECOGNIZED:{name}"
            client_socket.send(message.encode('utf-8'))
            print(f"Sent: {message}")
        except Exception as e:
            print(f"Error sending message: {e}")

In [6]:
if __name__ == "__main__":
    # Start the client connection
    client_socket = start_client()
    if not client_socket:
        exit(1)

    # Create LBPH Face Recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    # Load the trained model
    recognizer.read('trainer.yml')

    # Path to the Haar cascade file for face detection
    face_cascade_Path = "haarcascade_frontalface_default.xml"
    # Create a face cascade classifier
    faceCascade = cv2.CascadeClassifier(face_cascade_Path)

    # Font for displaying text on the image
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Initialize user IDs and associated names
    names = ['None']
    with open('names.json', 'r') as fs:
        names = json.load(fs)
        names = list(names.values())

    # Video Capture from the default camera (camera index 0)
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)  # Set width
    cam.set(4, 480)  # Set height

    # Minimum width and height for the window size to be recognized as a face
    minW = 0.1 * cam.get(3)
    minH = 0.1 * cam.get(4)

    # Create the window
    cv2.namedWindow('camera', cv2.WINDOW_NORMAL)
    # Resize the window
    cv2.resizeWindow('camera', 640, 480)  # Change the values as needed (e.g., 320, 240 for a smaller window)

    last_sent_time = time.time()  # Initialize the timer
    
    while True:
        name = ""
        # Read a frame from the camera
        ret, img = cam.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(int(minW), int(minH)),
        )

        for (x, y, w, h) in faces:
            # Draw a rectangle around the detected face
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Recognize the face using the trained model
            id, confidence = recognizer.predict(gray[y:y + h, x:x + w])

            if confidence > 10:
                try:
                    # Recognized face
                    name = names[id]
                    confidence_text = "  {0}%".format(round(confidence))
                except IndexError:
                    name = "Who are you?"
                    confidence_text = "N/A"
            else:
                # Unknown face
                name = "Who are you?"
                confidence_text = "N/A"

            # Check if 5 seconds have passed since the last recognition
            current_time = time.time()
            if current_time - last_sent_time >= 5:
                # Send the recognized name to the server
                send_recognized_name(name, client_socket)
                last_sent_time = current_time  # Update the timer

            # Display the recognized name and confidence level on the image
            cv2.putText(img, name, (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, confidence_text, (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)

        # Display the image with rectangles around faces
        cv2.imshow('camera', img)

        # Press Escape or 'q' to exit the webcam/program
        k = cv2.waitKey(10) & 0xff
        if k == 27 or k == ord('q'):  # 27 is the Escape key; ord('q') checks for 'q'
            break

    print("\n [INFO] Exiting Program.")
    # Release the camera
    cam.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()


Connected to the server.


Exception in thread Thread-3 (receive_messages):
Traceback (most recent call last):
  File "c:\Users\medoa\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\medoa\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\medoa\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\medoa\AppData\Local\Temp\ipykernel_4868\1331520512.py", line 12, in receive_messages
NameError: name 'breaks' is not defined


Sent: SENDTO:1|RECOGNIZED:MohammedAdnan
Received from server: Message received!
Sent: SENDTO:1|RECOGNIZED:MohammedAdnan
Sent: SENDTO:1|RECOGNIZED:MohammedAdnan

 [INFO] Exiting Program.
